In [ ]:
from machinedesign.viz import grid_of_images_default
from machinedesign.autoencoder.interface import load
import keras.backend as K
import numpy as np
from skimage.io import imsave
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA, FactorAnalysis, KernelPCA
from IPython.display import Image
from sklearn.neighbors import KernelDensity
from machinedesign.viz import horiz_merge
from functools import partial

from sklearn.decomposition import DictionaryLearning
def get_thresholds(X, X_true):
    ratio = (X_true > 0).mean(axis=(0, 2, 3))
    th = []
    for i in range(X.shape[1]):
        values = X[:, i].flatten()
        tpl = np.linspace(values.min(), values.max(), 10)
        for v in tpl:
            if (values>v).mean() < ratio[i]:
                break
        th.append(v)
    return np.array(th)

def minmax(X, X_true):
    a = X.min(axis=(0, 2, 3), keepdims=True)
    b = X.max(axis=(0, 2, 3), keepdims=True)
    X = (X - a) / (b - a + 1e-7)
    a = X_true.min(axis=(0, 2, 3), keepdims=True)
    b = X_true.max(axis=(0, 2, 3), keepdims=True)
    return a + X * (b - a)

def gaussian_kernel(A, B, sigmas=[1]):
    A_ = A[:, None, :]
    B_ = B[None, :, :]
    out = 0
    for sigma in sigmas:
        m = ((A_ - B_) ** 2).sum(axis=2)
        out += K.exp(-m / (2 * sigma**2))
    return out.sum()

def mmd(X, Y, kernel=gaussian_kernel):
    X_ = X
    Y_ = Y
    a = kernel(X_, X_)
    b = kernel(X_, Y_)
    c = kernel(Y_, Y_)
    N = X_.shape[0]
    M = Y_.shape[0]
    mmd_sqr = 1. / (N**2) * a - (2. / (N * M)) * b + (1. / M**2) * c
    return mmd_sqr

In [ ]:
!ls ../results/

In [ ]:
folder = 'flaticon'

In [ ]:
model = load('../results/{}'.format(folder))

In [ ]:
data = np.load('../../data/shoes.npz')
X = data['X']
X = X / 255.

In [ ]:
model.summary()

In [ ]:
O = []
acts = ('input_1', 'activation_1', 'activation_2', 'activation_3', 'activation_4', 'activation_5', 'activation_6')
for j, act in enumerate(acts):
    inp = model.layers[0].input
    hid_layer = model.get_layer(act)
    hid = hid_layer.output
    out = model.layers[-1].output
    enc = K.function([inp], hid)
    dec = K.function([hid], out)

    hid = enc([X[0:9]])
    print(hid.shape)
    out = []
    space = np.logspace(-1, 1, 10)
    for i, b in enumerate(space):
        noise = np.random.normal(0, b, size=hid.shape)
        y = dec([hid + noise])
        y = grid_of_images_default(y)
        out.append(y)
    out = np.array(out)    
    out = grid_of_images_default(out, shape=(len(space), 1))
    O.append(out)
O = np.array(O)
O = grid_of_images_default(O, shape=(1, len(acts)))
imsave('out.png'.format(j), O)

In [ ]:
inp = model.layers[0].input
hid_layer = model.get_layer('activation_3')
hid = hid_layer.output
out = model.layers[-1].output
enc = K.function([inp], hid)
dec = K.function([hid], out)

In [ ]:
hid = enc([X[0:100]])
hid_flat = hid.reshape((hid.shape[0],-1))
out = []
for i, b in enumerate(np.logspace(-1, 1.5, 16)):
    k = KernelDensity(bandwidth=b, kernel='gaussian')
    k.fit(hid_flat)
    g = k.sample(100)
    g = g.reshape((g.shape[0],) + hid.shape[1:])
    #g = minmax(g, hid)
    #th = get_thresholds(g, hid_orig)
    #th = th[None, :, None, None]
    g = g*(g > 0)
    y = dec([g])
    #y = (y>0.5)*255
    im = grid_of_images_default(y)
    out.append(im)
    #imsave('out{}.png'.format(i), im)
out = np.array(out)    
out = grid_of_images_default(out)
imsave('out.png', out)

In [ ]:
inp = model.layers[0].input
hid_layer = model.get_layer('activation_3')
hid = hid_layer.output
out = model.layers[-1].output
enc = K.function([inp], hid)
dec = K.function([hid], out)

h = enc([X[0:64]])
print(h.shape)
h[:]=0.
for i in range(h.shape[0]):
    h[i, i, 8, 8]  = 100
y= dec([h])
print(y.shape)
y = grid_of_images_default(y)
imsave('out.png', y)

In [ ]:
data_test = np.load('../../data/hwrt.npz')
X_test = data_test['X'] / 255.

In [ ]:
inp = model.layers[0].input
hid_layer = model.get_layer('activation_3')
hid = hid_layer.output
out = model.layers[-1].output
enc = K.function([inp, K.learning_phase()], hid)
dec = K.function([hid, K.learning_phase()], out)

In [ ]:
I = X_test[0:100]
x = grid_of_images_default(I)
hid = enc([I, 0])
y = dec([hid, 0])
y = grid_of_images_default(y)
out = horiz_merge(x, y)
imsave('out.png', out)